In [ ]:
%pip install --upgrade setuptools
%pip install --upgrade gradio
%pip install ipywidgets
%pip install openai
%pip install pandas
%pip install tabulate
%pip install python-dotenv

In [ ]:
%pip install -e ../

In [ ]:
import json
import setuptools

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from dotenv import dotenv_values
from typechat import Failure, TypeChatJsonTranslator, TypeChatValidator, create_language_model
from examples.music import schema as music
from examples.music.client import handle_call, get_client_context

In [ ]:
env_vals = dotenv_values()
model = create_language_model(env_vals)
validator = TypeChatValidator(music.PlayerActions)
translator = TypeChatJsonTranslator(model, validator, music.PlayerActions)
player_context = await get_client_context(env_vals)

In [ ]:
import pandas

async def get_translation(message, history):
    result = await translator.translate(message)
    if isinstance(result, Failure):
        return f"Translation Failed ❌ \n Context: {result.message}"
    else:
        df = pandas.DataFrame.from_dict(result["actions"])
        try:
            for action in result["actions"]:
                await handle_call(action, player_context)
            return f"Translation Succeeded! ✅\n Table View \n ``` {df.fillna('').to_markdown(tablefmt='grid')} \n ```  \n"
        except Exception as error:
            return f"An exception occurred: {error}"
        

def get_examples():
    example_prompts = []
    with open('../examples/music/input.txt') as prompts_file:
        for line in prompts_file:
            example_prompts.append(line)
    return example_prompts

In [ ]:
import gradio as gr

gr.ChatInterface(get_translation, title="🎵 Music", examples=get_examples()).launch()
